# Распознавание изображений с данными семисегментных индикаторов

In [259]:
import imutils  # pip install imutils
from imutils.perspective import four_point_transform
from imutils import contours

import cv2 as cv     # pip install opencv-python
from matplotlib import pyplot as plt

# Описание включения сегментов для разных цифр
# порядок сверху вниз, слева направо
DIGITS_LOOKUP = {
    (1, 1, 1, 0, 1, 1, 1): 0,
    (0, 0, 1, 0, 0, 1, 0): 1,
    (1, 0, 1, 1, 1, 1, 0): 2,
    (1, 0, 1, 1, 0, 1, 1): 3,
    (0, 1, 1, 1, 0, 1, 0): 4,
    (1, 1, 0, 1, 0, 1, 1): 5,
    (1, 1, 0, 1, 1, 1, 1): 6,
    (1, 0, 1, 0, 0, 1, 0): 7,
    (1, 1, 1, 1, 1, 1, 1): 8,
    (1, 1, 1, 1, 0, 1, 1): 9
}

In [260]:
# Для начала считаем имеющееся изображение весов
image = cv.imread("shot.jpg")
image = imutils.resize(image, height=400)

# Перевод в оттенки серого
grey = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
cv.imwrite("grey_shot.jpg", grey)

# Гаусово размытие с ядром 5х5 для удаления мелкого шума
blur = cv.GaussianBlur(grey, (5, 5), 0)
cv.imwrite("blur_shot.jpg", blur)

# Определение границ при помощи алгоритма Кэнни
edge = cv.Canny(blur, 20, 100,  1000)
cv.imwrite("edge_shot.jpg", edge)

True

В результате на изображении получаем четко очерченные границы дисплея.

In [264]:
# Найдем контуры и выделим ЖК-дисплей

# Создаем карту границ
cnts = cv2.findContours(edge.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
# Сортируем границы по внутренней площади ими ограничиваемой
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

displayCnt = None
for c in cnts:
    # Аппроксимируем контуры
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # Если контур содержит четыре вершины, 
    # предполагаем, что это искомый прямоугольник ЖК-индикатора
    if len(approx) == 4:
        displayCnt = approx
        break

In [265]:
LCD = four_point_transform(grey, displayCnt.reshape(4, 2))
cv2.imwrite("LCD.jpg", LCD)

True

Теперь необходимо на изображении ЖК-дисплея выделить непосредственно сами цифры.

In [266]:
thresh = cv2.threshold(LCD, 65, 255, cv2.THRESH_BINARY)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 15))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
cv2.imwrite("thresh.jpg", thresh)

True

In [275]:
from PIL import Image
from pytesseract import image_to_string

In [279]:
print(image_to_string(Image.open("thresh.jpg")))

149.1
